<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [51]:
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')

In [53]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [54]:
# текст запроса
query_3_1 = f'''
select count(*)
from public.vacancies
'''

In [55]:
# результат запроса
df_vacancies = pd.read_sql_query(query_3_1, connection)
df_vacancies


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [56]:
# текст запроса
query_3_2 = f'''
select count(*)
from public.employers
'''

In [57]:
df_employers = pd.read_sql_query(query_3_2, connection)
df_employers
# результат запроса

,count
0,23501


3. Посчитайте с помощью запроса количество регионов (таблица areas).

In [58]:
# текст запроса
query_3_3 = f'''
select count(*)
from public.areas
'''

In [59]:
# результат запроса
df_areas = pd.read_sql_query(query_3_3, connection)
df_areas

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [60]:
# текст запроса
query_3_4 = f'''
select count(*)
from public.industries
'''

In [61]:
# результат запроса
df_industries = pd.read_sql_query(query_3_4, connection)
df_industries

,count
0,294


#### Выводы по предварительному анализу данных:
1. В среднем каждый работодатель предоставляет по 2 вакансии.
2. Среднее количество сфер деятельности указаных работодателем 4,6.

***

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [62]:
# текст запроса
query_4_1 = f'''
select count(v.id),
a.name
from public.vacancies v 
    join public.areas a on v.area_id=a.id 
group by a.name
order by 1 desc
limit 5
'''

In [63]:
# результат запроса
df_vac_area = pd.read_sql_query(query_4_1, connection)
df_vac_area

,count,name
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [64]:
# текст запроса
query_4_2 = f'''
select count(*)
from public.vacancies 
where salary_from IS NOT NULL or salary_to IS NOT NULL
'''

In [65]:
# результат запроса
df_salary = pd.read_sql_query(query_4_2, connection)
df_salary

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [66]:
# текст запроса
query_4_3 = f'''
select avg(salary_from),
avg(salary_to)
from public.vacancies
'''

In [67]:
# результат запроса
df_avg_salary = pd.read_sql_query(query_4_3, connection)
df_avg_salary.round()

,avg,avg
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [68]:
# текст запроса
query_4_4 = f'''
select concat(schedule, ' - ', employment) se,
count(id)
from public.vacancies 
group by se
order by 2 desc
limit 2
'''



In [69]:
# результат запроса
df_se = pd.read_sql_query(query_4_4, connection)
df_se

,se,count
0,Полный день - Полная занятость,35367
1,Удаленная работа - Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [70]:
# текст запроса
query_4_5 = f'''
select experience,
count(id)
from public.vacancies 
group by 1
order by 2
limit 5
'''

In [71]:
# результат запроса
df_exp = pd.read_sql_query(query_4_5, connection)
df_exp

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


#### Выводы по детальному анализу вакансий:
1. Верхняя вилка заработной платы в среднем в 1.5 раза больше нижней вилки.
2. Количество вакансий в Москве максимальное.
3. Примерно половина вакансий с требуемым опытом от 1 до 3х лет.


***

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [72]:
# текст запроса
query_5_1 = f'''
select e.name,
count(v.id)
from public.vacancies v 
    join public.employers e on v.employer_id=e.id
group by 1
order by 2 desc
limit 5
'''

In [73]:
# результат запроса
df_emp = pd.read_sql_query(query_5_1, connection)
df_emp

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [74]:
# текст запроса
query_5_2 = f'''
select a.name,
	count(distinct e.id) employers, 
	count(distinct v.id) vacancies
from public.areas a 
	left join public.employers e on e.area = a.id
    left join public.vacancies v on v.area_id=a.id
where v.id is null
group by 1 
order by 3 asc, 2 desc
limit 5
'''

In [75]:
# результат запроса
df_ev = pd.read_sql_query(query_5_2, connection)
df_ev

,name,employers,vacancies
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [76]:
# текст запроса
query_5_3 = f'''
select e.name,
	count(distinct v.area_id)
from public.vacancies v 
	join public.employers e on e.id = v.employer_id
group by 1
order by 2 desc
limit 5
'''

In [77]:
# результат запроса
df_ea = pd.read_sql_query(query_5_3, connection)
df_ea

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [78]:
# текст запроса
query_5_4 = f'''
select count(*)
from public.employers e
	left join public.employers_industries ei on ei.employer_id=e.id
where ei.industry_id is null
'''

In [79]:
# результат запроса
df_ei = pd.read_sql_query(query_5_4, connection)
df_ei

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [80]:
# текст запроса
query_5_5 = f'''
select e.name,
count(ei.industry_id) industries_count
from public.employers e
	left join public.employers_industries ei on ei.employer_id=e.id
group by 1
having count(ei.industry_id) = 4
order by 1
limit 3
'''

In [81]:
# результат запроса
df_ic = pd.read_sql_query(query_5_5, connection)
df_ic

,name,industries_count
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [82]:
# текст запроса
query_5_6 = f'''
select count(*)
from public.employers_industries ei
	left join public.industries i on ei.industry_id=i.id
where i.name = 'Разработка программного обеспечения'
'''

In [83]:
# результат запроса
df_ei = pd.read_sql_query(query_5_6, connection)
df_ei

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [86]:
# код для получения списка городов-милионников
from bs4 import BeautifulSoup
import requests

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-'\
    '%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%'\
    'BE%D1%81%D1%81%D0%B8%D0%B8'
pages = requests.get(url)
pages.text
page = BeautifulSoup(pages.text, 'html.parser')
tabs = page.find_all('tbody')
cities = [tab.text for tab in tabs[1].find_all('a')[4::]]
cities.remove('[a]')
display(cities)


['Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Челябинск',
 'Красноярск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [ ]:
# текст запроса
query_5_7 = f'''
select a.name area,
count(v.id) vac_cnt
from public.vacancies v
	join public.areas a on a.id=v.area_id
	join public.employers e on e.id=v.employer_id
where e.name='Яндекс'
group by a.name
order by 2
'''

In [ ]:
# результат запроса
df_yandex = pd.read_sql_query(query_5_7, connection)
df_yandex['area'] = df_yandex['area'].astype('string')
df_yandex = df_yandex[df_yandex['area'].isin(cities)]
df_yandex = df_yandex.append(
    {'area':'Total', 'vac_cnt':df_yandex['vac_cnt'].sum()}, ignore_index=True) 
display(df_yandex)

,area,vac_cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


#### Выводы по анализу работодателей:
1. Яндекс крупнейший (с большим отрывом) работодатель СНГ предоставляющий вакансии в большинстве регионов.
2. Для выводов требуется более глубокий анализ работодателей чем предусмотрен шаблоном.
3. Поляков Денис Иванович предоставляет вакансии для жителей аж 88 регионов.=) Молодец!)

***

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса
query_6_1 = f'''
select count(*)
from  public.vacancies v 
where lower(v.name) like '%data%' or lower(v.name) like '%данн%'
'''

In [ ]:
# результат запроса
df_d = pd.read_sql_query(query_6_1, connection)
df_d 

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса
query_6_2 = f'''
select *
from  public.vacancies v 
where (lower(v.name) like '%data scientist%' 
	or lower(v.name) like '%data science%'
	or lower(v.name) like '%исследователь данных%'
	or (v.name like '%ML%' and v.name not like '%HTML%')
	or lower(v.name) like '%machine learning%'
	or lower(v.name) like '%машинн%обучен%')
	and lower(v.name) like '%junior%'
union 
select *
from  public.vacancies v
where (lower(v.name) like '%data scientist%' 
	or lower(v.name) like '%data science%'
	or lower(v.name) like '%исследователь данных%'
	or (v.name like '%ML%' and v.name not like '%HTML%')
	or lower(v.name) like '%machine learning%'
	or lower(v.name) like '%машинн%обучен%') 
	and lower(experience) like '%нет опыта%' or lower(schedule) = '%cтажировка%'
'''

In [ ]:
# результат запроса
df_dsj = pd.read_sql_query(query_6_2, connection)
df_dsj.shape

(51, 10)

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса
query_6_3 = f'''
select *
from  public.vacancies v 
where (lower(v.name) like '%data scientist%' 
	or lower(v.name) like '%data science%'
	or lower(v.name) like '%исследователь данных%'
	or (v.name like '%ML%' and v.name not like '%HTML%')
	or lower(v.name) like '%machine learning%'
	or lower(v.name) like '%машинн%обучен%')
	and (key_skills like '%SQL%' or key_skills like '%Postgres%')
'''

In [ ]:
# результат запроса
df_dssp = pd.read_sql_query(query_6_3, connection)
df_dssp.shape

(201, 10)

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса
query_6_4 = f'''
select *
from  public.vacancies v 
where (lower(v.name) like '%data scientist%' 
	or lower(v.name) like '%data science%'
	or lower(v.name) like '%исследователь данных%'
	or (v.name like '%ML%' and v.name not like '%HTML%')
	or lower(v.name) like '%machine learning%'
	or lower(v.name) like '%машинн%обучен%')
	and (key_skills like '%Python%')
'''

In [ ]:
# результат запроса
df_dsp = pd.read_sql_query(query_6_4, connection)
df_dsp

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,50533416,Junior+ CV/ML developer,Python\tSQL\tРабота в команде\tOpenCV\tML,Гибкий график,От 1 года до 3 лет,Полная занятость,NaN,NaN,4,80190
1,54824572,Data Scientist,Oracle Pl/SQL\tБазы данных\tMachine Learning\t...,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,160,111304
2,54825318,Data Scientist (Junior+),Python\tOpenCV\tTensorflow\tComputer Vision\tН...,Полный день,От 1 года до 3 лет,Полная занятость,50000.0,70000.0,72,215942
3,66120847,AI/ML Senior Engineer,Python\tFlask\tLinux\tAWS\tGit,Гибкий график,От 3 до 6 лет,Полная занятость,NaN,NaN,2759,204511
4,55351147,Data Scientist,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,53,3127
...,...,...,...,...,...,...,...,...,...,...
346,55478561,Machine Learning and Computer vision developer,Python\tLinux\tSQL\tPyTorch\tOpenCV\tKeras\tCa...,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,2,6096024
347,55306481,Data scientist / Исследователь-программист данных,Python\tАналитическое мышление\tАнализ данных\...,Полный день,От 1 года до 3 лет,Полная занятость,115000.0,172000.0,1,4846045
348,54918116,Специалист ML/DS,Python\tNumpy\tPyTorch\tPandas\tМатематический...,Полный день,От 3 до 6 лет,Полная занятость,61024.0,73228.0,160,5969955
349,55185186,Data Scientist в области рекомендательных систем,Python\tScikit-learn\tPandas\tGit\tNumpy,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,66,3529


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [102]:
# текст запроса
query_6_5 = f'''
select round(avg(length(key_skills) - length(replace(key_skills, CHR(9),'')) + 1), 2)
from  public.vacancies v 
where (lower(v.name) like '%data scientist%' 
	or lower(v.name) like '%data science%'
	or lower(v.name) like '%исследователь данных%'
	or (v.name like '%ML%' and v.name not like '%HTML%')
	or lower(v.name) like '%machine learning%'
	or lower(v.name) like '%машинн%обучен%')
	and lower(key_skills) is not null
'''


In [103]:
# результат запроса
df_ks = pd.read_sql_query(query_6_5, connection)
df_ks

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [105]:
# текст запроса
query_6_6 = f'''
select experience,
round(avg(coalesce((salary_from+salary_to)/2, salary_to, salary_from)))
from  public.vacancies v 
where (lower(v.name) like '%data scientist%' 
	or lower(v.name) like '%data science%'
	or lower(v.name) like '%исследователь данных%'
	or (v.name like '%ML%' and v.name not like '%HTML%')
	or lower(v.name) like '%machine learning%'
	or lower(v.name) like '%машинн%обучен%')
	and (salary_from is not null or salary_to is not null)
group by experience
'''

In [106]:
# результат запроса
df_ds_salary = pd.read_sql_query(query_6_6, connection)
df_ds_salary

,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


#### Выводы по предметному анализу:
1. aaa
2. fff
3. add

***

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

In [ ]:
connection.close()